# Создание дашборда по пользовательским событиям для агрегатора новостей Яндекс.Дзен

## Описание проекта

Работая аналитиком в Яндекс.Дзене. Почти всё рабочее время занимает анализ пользовательского взаимодействия с карточками статей.

Каждую карточку определяют её тема и источник (у него тоже есть тема). Примеры тем: «Красота и здоровье», «Россия», «Путешествия». Пользователей системы характеризует возрастная категория. Скажем, «26-30» или «45+».

Есть три способа взаимодействия пользователей с системой:

1) Карточка отображена для пользователя (show);

2) Пользователь кликнул на карточку (click);

3) Пользователь просмотрел статью карточки (view).

Каждую неделю менеджеры задают одни и те же вопросы:

1. Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
2. Как много карточек генерируют источники с разными темами?
3. Как соотносятся темы карточек и темы источников?

На шестую неделю работы решено, что процесс пора автоматизировать. Нужно сделать дашборд. Дашборд будет основываться на пайплайне, который будет брать данные из таблиц, в которых хранятся сырые данные, трансформировать данные и укладывать их в агрегирующую таблицу. Пайплайн будет разработан дата-инженерами.

# Написание ТЗ

Пообщавшись с менеджерами и администраторами баз данных, было написано  краткое ТЗ:

1) Бизнес-задача: анализ взаимодействия пользователей с карточками Яндекс.Дзен;

2) Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;

3) Кто будет основным пользователем дашборда: менеджеры по анализу контента;

4) Состав данных для дашборда:

- История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
- Разбивка событий по темам источников;
- Таблица соответствия тем источников темам карточек;

5) По каким параметрам данные должны группироваться:

- Дата и время;
- Тема карточки;
- Тема источника;
- Возрастная группа;

6) Характер данных:

- История событий по темам карточек — абсолютные величины с разбивкой по минутам;
- Разбивка событий по темам источников — относительные величины (% событий);
- Соответствия тем источников темам карточек - абсолютные величины;

7) Важность: все графики имеют равную важность;

8) Источники данных для дашборда: cырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw);

9) База данных, в которой будут храниться агрегированные данные: дополнительные агрегированные таблицы в БД zen;

10) Частота обновления данных: один раз в сутки, в полночь по UTC;

# Макет дашборда

<img align="center" src="dashboard_maket.png" height="" width="">

# Запрос на коннекцию к базе

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

query = '''
        SELECT * FROM dash_visits 
        '''
 
dash_visits = pd.io.sql.read_sql(query, con = engine) 

# сохраним выгрузку в корневой папке, назовём dash_visits.csv
dash_visits.to_csv('dash_visits.csv')

In [2]:
# посмотрим на данные
display(dash_visits.head())

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


In [3]:
# проверим корректность данных
display(dash_visits.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


None

# Изготовление и выгрузка дашборда в Tableau Public

В Tableau Public на основе файла dash_visits.csv сформируем дашборд в соответствии с макетом, описанным выше:
https://public.tableau.com/app/profile/victor1223/viz/Praktikum_dashdoard_16418182557970/_?publish=yes

# Результаты работы и ответы на три ключевых вопроса проекта:

https://disk.yandex.ru/i/-st_4AgtxSYGZw